In [1]:
import json
import os
import pygarment.data_config as data_config
from pathlib import Path
from pygarment.meshgen.sim_config import PathCofig
from pygarment.meshgen.boxmeshgen import BoxMesh
from pygarment.meshgen.simulation import run_sim
from tqdm import tqdm

Warp 1.0.0-beta.6 initialized:
   CUDA Toolkit: 12.4, Driver: 12.4
   Devices:
     "cpu"    | x86_64
     "cuda:0" | NVIDIA A100-SXM4-80GB (sm_80)
     "cuda:1" | NVIDIA A100-SXM4-80GB (sm_80)
     "cuda:2" | NVIDIA A100-SXM4-80GB (sm_80)
     "cuda:3" | NVIDIA A100-SXM4-80GB (sm_80)
     "cuda:4" | NVIDIA A100-SXM4-80GB (sm_80)
     "cuda:5" | NVIDIA A100-SXM4-80GB (sm_80)
     "cuda:6" | NVIDIA A100-SXM4-80GB (sm_80)
     "cuda:7" | NVIDIA A100-SXM4-80GB (sm_80)
   Kernel cache: /data/lixinag/.cache/warp/1.0.0-beta.6


In [2]:
PROJECT_ROOT = '/data/lixinag/project/GarmentCode'
DATA_ROOT = f'{PROJECT_ROOT}/A_DATA/GarmentCodeData'
SPLIT_FILE_PATH = f'{DATA_ROOT}/train_test_split.json'
SIM_CONFIG_PATH = f'{PROJECT_ROOT}/assets/Sim_props/default_sim_props_modified.yaml'
OUTPUT_PATH = f'{PROJECT_ROOT}/simulation_output/'
BODY_DEFAULT_DIR = f'{PROJECT_ROOT}/A_RESULT/bodies'
BODY_NAME = 'SMPL_FEMALE_A_POSE_110_ORI'

In [3]:
def get_input_file_path_pool(split_file_path, end_with='_specification.json'):
    input_file_path_pool = []
    with open(split_file_path, 'r') as f:
        data = json.load(f)
        for key_name in data.keys():
            for file_path in data[key_name]:
                file_id = file_path.split('/')[-1]
                input_file_path_pool.append(Path(os.path.join(file_path, file_id + end_with)))
    return input_file_path_pool

input_file_path_pool = get_input_file_path_pool(SPLIT_FILE_PATH)

In [4]:
props = data_config.Properties(SIM_CONFIG_PATH)
props.set_section_stats('sim', fails={}, sim_time={}, spf={}, fin_frame={}, body_collisions={}, self_collisions={})
props.set_section_stats('render', render_time={})

In [5]:
input_file_path_pool[0:3]

[PosixPath('garments_5000_0/default_body/rand_RVS1QWPXD0/rand_RVS1QWPXD0_specification.json'),
 PosixPath('garments_5000_0/default_body/rand_M4Z0YOEJ6V/rand_M4Z0YOEJ6V_specification.json'),
 PosixPath('garments_5000_0/default_body/rand_E489IFV849/rand_E489IFV849_specification.json')]

In [ ]:
t = Path(DATA_ROOT) / input_file_path_pool[0]


In [6]:
for spec_path in tqdm(input_file_path_pool):
    garment_name, _, _ = spec_path.stem.rpartition('_')
    paths = PathCofig(
        data_root_path=DATA_ROOT,
        in_element_path=spec_path.parent,  
        out_path=OUTPUT_PATH, 
        in_name=garment_name,
        out_name=spec_path.parent,
        body_default_dir=BODY_DEFAULT_DIR,
        body_name=BODY_NAME,
        smpl_body=True)

    garment_box_mesh = BoxMesh(paths.in_g_spec, props['sim']['config']['resolution_scale'])
    garment_box_mesh.load()
    garment_box_mesh.serialize(
        paths, store_panels=False, uv_config=props['render']['config']['uv_texture'])

    props.serialize(paths.element_sim_props)

    run_sim(
        garment_box_mesh.name, 
        props, 
        paths,
        save_v_norms=False,
        store_usd=False,  # NOTE: False for fast simulation!
        optimize_storage=False,
        verbose=False
    )

    break

garment_name, paths

  0%|          | 0/115195 [00:00<?, ?it/s]

BoxMesh::rand_RVS1QWPXD0::WARNING::Path does not exist: /data/lixinag/project/GarmentCode/A_DATA/GarmentCodeData/garments_5000_0/default_body/rand_RVS1QWPXD0/design_params.yaml
Module warp.sim.inertia load on device 'cuda:0' took 37.22 ms
Module warp.collision.panel_assignment load on device 'cuda:0' took 6.58 ms
sl_right_cuff_skirt_b:right_arm
sl_right_cuff_skirt_f:right_arm
right_sleeve_b:right_arm
right_sleeve_f:right_arm
right_btorso:body
right_ftorso:body
skirt_back:legs
skirt_front:legs
left_btorso:body
left_ftorso:body
left_sleeve_b:left_arm
left_sleeve_f:left_arm
sl_left_cuff_skirt_b:left_arm
sl_left_cuff_skirt_f:left_arm
Using attachment for lower_interface with 92 vertices
Using attachment for right_collar with 28 vertices
Using attachment for left_collar with 28 vertices
Module warp.sim.utils load on device 'cuda:0' took 16.26 ms
Module warp.utils load on device 'cuda:0' took 5.83 ms
Module warp.sim.model load on device 'cuda:0' took 2.35 ms
Module warp.sim.collide load on d

  0%|          | 0/115195 [00:56<?, ?it/s]

Rendering rand_RVS1QWPXD0 took 1.720266342163086s

Simulation pipeline took: 0 m 47.869 s


('rand_RVS1QWPXD0', <pygarment.meshgen.sim_config.PathCofig at 0x7fd3301c0c10>)